# Core Walkthrough: Categories, Functors, Naturality, Monads

This notebook demonstrates non-agent features:
- Build small categories (terminal, discrete, simplex Δ³)
- Define and check functors
- Define and check natural transformations
- Use FP monads (`Id`, `Maybe`) and `Kleisli`
- Render Mermaid diagrams (categories, functors, naturality, plan)



In [ ]:
from LambdaCat.core.standard import terminal_category, discrete, simplex, walking_isomorphism
from LambdaCat.core.laws import run_suite
from LambdaCat.core.laws_category import CATEGORY_SUITE

Term = terminal_category()
Disc = discrete(["A","B"]) 
Delta3 = simplex(3)
Iso = walking_isomorphism()

assert run_suite(Term, CATEGORY_SUITE).ok
assert run_suite(Disc, CATEGORY_SUITE).ok
assert run_suite(Delta3, CATEGORY_SUITE).ok
assert run_suite(Iso, CATEGORY_SUITE).ok

Term, Disc, Delta3, Iso


In [ ]:
from LambdaCat.core.functor import FunctorBuilder
from LambdaCat.core.laws_functor import FUNCTOR_SUITE

F = (FunctorBuilder('F', source=Delta3, target=Iso)
     .on_objects({"0":"A","1":"A","2":"B","3":"B"})
     .on_morphisms({"0->1":"id:A","1->2":"f","2->3":"id:B","0->3":"f"})
     .build())

report = run_suite(F, FUNCTOR_SUITE)
print(report.to_text())
F.name, len(F.object_map), len(F.morphism_map)


In [ ]:
from LambdaCat.core.natural import Natural
from LambdaCat.core import NATURAL_SUITE

eta = Natural(source=F, target=F, components={"0":"id:A","1":"id:A","2":"id:B","3":"id:B"})
report = run_suite(eta, NATURAL_SUITE)
print(report.to_text())
list(eta.components.items())


In [ ]:
from LambdaCat.core.fp.kleisli import Kleisli
from LambdaCat.core.fp.instances.maybe import Maybe
from LambdaCat.core.fp.instances.identity import Id

parse_int = Kleisli(lambda s: Maybe(int(s)) if str(s).isdigit() else Maybe(None))
recip = Kleisli(lambda n: Maybe(None) if n == 0 else Maybe(1.0 / n))
pipeline = parse_int.then(recip)

print(pipeline.run("12"))   # Maybe(1/12)
print(pipeline.run("oops"))  # Maybe(None)

# Identity monad
inc = Kleisli(lambda n: Id(n + 1))
dbl = Kleisli(lambda n: Id(n * 2))
print(inc.then(dbl).run(3))  # Id(8)



In [ ]:
from LambdaCat.extras.viz_mermaid import render_all, TwoCellView, category_mermaid, functor_mermaid, naturality_mermaid

md = render_all({
    'Delta3': Delta3,
    'Iso': Iso,
    'F': F,
    'eta': eta,
}, out_dir=None)

print(md['Delta3__category.md'][:200])
print(md['F__functor.md'][:200])



## Opposite category and dual functors
We can construct the opposite category `C^op` and examine how a functor `F: C -> D` induces structure on `C^op`.


In [ ]:
from LambdaCat.core.ops_category import opposite_category

Delta3_op = opposite_category(Delta3)
print(len(Delta3.objects), len(Delta3.arrows))
print(len(Delta3_op.objects), len(Delta3_op.arrows))


## Functor algebra: composition and identity
Demonstrate `FunctorBuilder` preserving identities and composition; verify with the law suite and inspect violations when broken.


In [ ]:
from LambdaCat.core.functor import CatFunctor
from LambdaCat.core.laws_functor import FUNCTOR_SUITE

# Builder enforces laws and raises on invalid data. For a violations demo,
# construct a CatFunctor directly with an incomplete/incorrect morphism map.
broken = CatFunctor(
    name="B",
    source=Delta3,
    target=Iso,
    object_map={"0":"A","1":"A","2":"B","3":"B"},
    morphism_map={"0->1":"id:A"},  # intentionally incomplete
)
rep = run_suite(broken, FUNCTOR_SUITE)
print(rep.to_text())


In [ ]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


In [ ]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


In [ ]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


In [ ]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


In [ ]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


## Monads and Kleisli composition
We explore `Maybe` for partiality and `Id` for pure computation; show associativity via chaining.


In [ ]:
from LambdaCat.core.natural import Natural
from LambdaCat.core import NATURAL_SUITE
bad = Natural(source=eta.source, target=eta.target, components={**eta.components, "1": "f"})
print(run_suite(bad, NATURAL_SUITE).to_text())


In [ ]:
from LambdaCat.core.fp.kleisli import kleisli_identity
I = kleisli_identity(Id.pure)
print(I.run(5))  # Id(5)

# Associativity check (empirical)
from random import randint
f = Kleisli(lambda n: Id(n + 1))
g = Kleisli(lambda n: Id(n * 2))
h = Kleisli(lambda n: Id(n - 3))

for _ in range(3):
    x = randint(0, 10)
    left = f.then(g).then(h).run(x)
    right = f.then(g.then(h)).run(x)
    assert left == right
print("Kleisli associativity holds on samples.")


## Serialization and diagrams
Convert a presentation to a plain dict and render diagrams for papers or PRs.


In [ ]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])


In [ ]:
from LambdaCat.core.builder import obj, arrow, build_presentation
from LambdaCat.core.convert import to_dict

A, B = obj("A"), obj("B")
F = arrow("f","A","B")
P = build_presentation((A,B),(F,))
print(to_dict(P))

md_map = render_all({'Delta3': Delta3, 'Iso': Iso}, out_dir=None)
print(list(md_map.keys())[:4])
